# Fit Estimation

In [ ]:
# please provide your OPENAI_KEY
OPENAI_KEY = "sk-<your-openai-key>"

## Related Functions

In [ ]:
import json
import re
import random
import requests
import base64

def run_local_vision_request(text, image_urls, temperature=0):
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    api_key = OPENAI_KEY

    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": text},
            ]
        }
    ]

    for image_path in image_urls:
        # Getting the base64 string
        base64_image = encode_image(image_path)
        messages[0]["content"].append({
            "image_url": f"data:image/jpeg;base64,{base64_image}"
        })

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": 2048,
        "temperature": temperature,
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def run_test(name, description, prompt, image_urls, logger, times=1):
    reason_list = []
    score_list = []
    while len(score_list) < times:
        shuffled_list = image_urls.copy()
        random.shuffle(shuffled_list)
        try:
            response = run_local_vision_request(
                text=prompt, 
                image_urls=shuffled_list,
            )
            matched_content = re.search(r'```json([\s\S]*?)```', response["choices"][0]["message"]["content"])
            shuffled_score = json.loads(matched_content.group(1))
            score = shuffled_score.copy()
            for i in range(len(score)):
                score[i] = shuffled_score[shuffled_list.index(image_urls[i])]
            reason_list.append(response["choices"][0]["message"]["content"])
            score_list.append(score)
            logger.log(name, description, prompt, shuffled_list, response["choices"][0]["message"]["content"], score)
        except Exception as e:
            print(e)
    return reason_list

class Logger:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log(self, name, description, prompt, image_urls, response, score):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "image_urls": image_urls, "response": response, "score": score}
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        data.append(json_data_to_add)
        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)

logger = Logger("./fit_estimation.json")


def revise(file_path, name, description, image_urls):
    with open(file_path, "r") as f:
        data = json.load(f)

    for result in data:
        if result["name"] == name and result["description"] == description:
            matched_content = re.search(r'```json([\s\S]*?)```', result["response"])
            shuffle_score = json.loads(matched_content.group(1))
            score = shuffle_score.copy()
            for i in range(len(score)):
                score[i] = shuffle_score[result['image_urls'].index(image_urls[i])]
            result['score'] = score

    with open("./fit_estimation.json", 'w') as file:
        json.dump(data, file, indent=4)

## Experiment 1

### Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
TASK = 'Here are {number} scatterplots. You are instructed to estimate how well the data points fitted to the shown line. Your score ranges from 0 (very low fit) to 100 (very high fit), a perfect fit would be to have all data points on the line. Please list the scores and the reason in the order of the images. '
JSON_FORMAT = "Please give an additional scoring result in json format at the end of your answe, like ```json[score1, score2, ...]```. "

### Data Preparation

In [ ]:
file_path = "./experiment_material/stimuli_exp1+exp2/"
sd_list = ["exp1_sd05", "exp1_sd10","exp1_sd15","exp1_sd20","exp1_sd25",]
index_list = ["1", "2", "3", "4", "5", "6"]

### Task Execution(parallel experiment)

In [ ]:
name = "fit_estimation_experiment1"
for i in index_list:
    description = "parallel_experiment_" + i
    image_urls = [file_path + sd + "_" + i + ".png" for sd in sd_list]
    prompt = ROLE + \
        TASK.format(number=len(image_urls)) + \
        JSON_FORMAT + \
        "DON'T say sorry or you cannot. YOU CAN."
    run_test(
        name=name,
        description=description,
        prompt=prompt,
        image_urls=image_urls,
        logger=logger,
        times=5)

## Experiment 2

### Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
TASK = 'Here are {number} scatterplots. You are instructed to estimate how well the data points fitted to the shown line. Your score ranges from 0 (very low fit) to 100 (very high fit), a perfect fit would be to have all data points on the line. Please list the scores and the reason in the order of the images. '
JSON_FORMAT = "Please give an additional scoring result in json format at the end of your answe, like ```json[score1, score2, ...]```. "

### Data Preparation

In [ ]:
file_path = "./experiment_material/stimuli_exp1+exp2/"
sd_list = ["02", "04", "06", "08", "10", "12"]
type_list = ["neutral", "light_down", "light_up", "middle_down", "middle_up", "strong_down", "strong_up"]

### Task Execution(Same SD)

In [ ]:
name = "fit_estimation_experiment2"
for sd in sd_list:
    description = "parallel_experiment_same_sd_" + sd
    image_urls = [file_path + "exp2_sd" + sd + "_" + t + ".png" for t in type_list]
    prompt = ROLE + \
        TASK.format(number=len(image_urls)) + \
        JSON_FORMAT + \
        "DON'T say sorry or you cannot. YOU CAN."
    run_test(
        name=name,
        description=description,
        prompt=prompt,
        image_urls=image_urls,
        logger=logger,
        times=5)

## Additional Experience

### Compare SD with Decentering

In [ ]:
name = "extra_experience"
description = "compare_sd02_strong_sd04_neutral"
image_urls = ['./fit/stimuli_exp1+exp2/exp2_sd02_strong_up.png',
              './fit/stimuli_exp1+exp2/exp2_sd02_strong_down.png',
              './fit/stimuli_exp1+exp2/exp2_sd04_neutral.png',]
prompt = ROLE + \
    TASK.format(number=len(image_urls)) + \
    JSON_FORMAT + \
    "DON'T say sorry or you cannot. YOU CAN."
run_test(
    name=name,
    description=description,
    prompt=prompt,
    image_urls=image_urls,
    logger=logger,
    times=5)

In [ ]:
name = "extra_experience"
description = "compare_sd04_middle_sd06_neutral"
image_urls = ['./fit/stimuli_exp1+exp2/exp2_sd04_middle_up.png',
              './fit/stimuli_exp1+exp2/exp2_sd04_middle_down.png',
              './fit/stimuli_exp1+exp2/exp2_sd06_neutral.png',]
prompt = ROLE + \
    TASK.format(number=len(image_urls)) + \
    JSON_FORMAT + \
    "DON'T say sorry or you cannot. YOU CAN."
run_test(
    name=name,
    description=description,
    prompt=prompt,
    image_urls=image_urls,
    logger=logger,
    times=5)

In [ ]:
name = "extra_experience"
description = "compare_sd02_sd12"
up = ['./fit/stimuli_exp1+exp2/exp2_sd02_light_up.png',
      './fit/stimuli_exp1+exp2/exp2_sd02_middle_up.png',
      './fit/stimuli_exp1+exp2/exp2_sd02_strong_up.png',
      './fit/stimuli_exp1+exp2/exp2_sd12_light_up.png',
      './fit/stimuli_exp1+exp2/exp2_sd12_middle_up.png',
      './fit/stimuli_exp1+exp2/exp2_sd12_strong_up.png', 
      ]
down = ['./fit/stimuli_exp1+exp2/exp2_sd02_light_down.png',
      './fit/stimuli_exp1+exp2/exp2_sd02_middle_down.png',
      './fit/stimuli_exp1+exp2/exp2_sd02_strong_down.png',
      './fit/stimuli_exp1+exp2/exp2_sd12_light_down.png',
      './fit/stimuli_exp1+exp2/exp2_sd12_middle_down.png',
      './fit/stimuli_exp1+exp2/exp2_sd12_strong_down.png', 
      ]
netural = ['./fit/stimuli_exp1+exp2/exp2_sd02_neutral.png',
           './fit/stimuli_exp1+exp2/exp2_sd12_neutral.png',]

for i in range(10):
    description = "compare_sd02_sd12" + "_parallel_experiment_" + str(i + 1)
    image_urls = netural[0:1] + [random.choice([u, d]) for u, d in zip(up[0:3], down[0:3])] + netural[1:2] + [random.choice([u, d]) for u, d in zip(up[3:6], down[3:6])]
    prompt = ROLE + \
    TASK.format(number=len(image_urls)) + \
    JSON_FORMAT + \
    "DON'T say sorry or you cannot. YOU CAN."
    run_test(
        name=name,
        description=description,
        prompt=prompt,
        image_urls=image_urls,
        logger=logger,
        times=5)

### Potential Senario

In [ ]:
description = "scenario2_6image"
image_urls = [
    "./fit/images_for_scenario2/sd0.2_decenter0.png",
    "./fit/images_for_scenario2/sd0.2_decenter0.99_downwards.png",
    "./fit/images_for_scenario2/sd0.2_decenter0.99_upwards.png",
    "./fit/images_for_scenario2/sd0.4_decenter0.png",
    "./fit/images_for_scenario2/sd0.4_decenter0.99_downwards.png",
    "./fit/images_for_scenario2/sd0.4_decenter0.99_upwards.png",
]

prompt = ROLE + \
    TASK.format(number=len(image_urls)) + \
    JSON_FORMAT + \
    "DON'T say sorry or you cannot. YOU CAN."
run_test(
    name=name,
    description=description,
    prompt=prompt,
    image_urls=image_urls,
    logger=logger,
    times=10)